In [ ]:
'''
Esse eu acredito que seja o primeiro script que você disse, mas caso não seja me fala
'''

In [ ]:
import arcpy
import pandas
import geopandas
import time

map_name = "Map"
cntd = 0

# Obtenha um projeto do ArcGIS Pro
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Obtenha um mapa dentro do projeto
m = aprx.listMaps()[0]

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PLAK0219d8af8ac043908352cb6faa9b9903'

PLANET_TILE_URL_TEMPLATE = "https://tiles.planet.com/data/v1/{item_type}/{item_id}/{{z}}/{{x}}/{{y}}.png?api_key={API_KEY}"

# Add planet tiles from search results
for result in geojson['features']:
    # item_id is the geojson feature id
    item_id = result["id"]
    # item_type is in the geojson feature properties
    item_type = result["properties"]["item_type"]

    # Create the Planet tile URL for this specific item
    tile_url = PLANET_TILE_URL_TEMPLATE.format(item_id=item_id, item_type=item_type, API_KEY=API_KEY)

    print(tile_url)
    
    m.addDataFromPath(tile_url, 'AUTOMATIC')
    time.sleep(2)

In [ ]:
# Caso nâo funcione teste o seguinte fluXo:

In [ ]:
# Stockton, CA bounding box (created via geojson.io)
geojson_geometry = {
  "type": "Polygon",
  "coordinates": 
     [
          [
            [
              -43.29520982852239,
              -22.94525882116791
            ],
            [
              -43.29520982852239,
              -22.991522610411707
            ],
            [
              -43.24091986050328,
              -22.991522610411707
            ],
            [
              -43.24091986050328,
              -22.94525882116791
            ],
            [
              -43.29520982852239,
              -22.94525882116791
            ]
          ]
        ]
}


input_dict = {
    'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {
        0: '<POLYGON ((-43.767 -22.913, -43.768 -22.931, -43.748 -22.932, -43.748 -22.913, -43.767 -22.913))>'
    }
}

# Extrair a string do polígono do input_dict
polygon_wkt = input_dict['config'][0]

# Limpar a string (remover caracteres indesejados)
cleaned_wkt = polygon_wkt.strip("<>").replace("POLYGON ((", "").replace("))", "")

# Dividir os pontos e converter em coordenadas [[lon, lat], ...]
coordinates = [
    [float(coord.split()[0]), float(coord.split()[1])] 
    for coord in cleaned_wkt.split(", ")
]

# Fechar o polígono (último ponto igual ao primeiro)
if coordinates[0] != coordinates[-1]:
    coordinates.append(coordinates[0])

# Estruturar no formato GeoJSON
geojson_geometry = {
    "type": "Polygon",
    "coordinates": [coordinates]  # Lista de anéis (no caso, 1 anel)
}

# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}


# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2024-08-28T00:00:00.000Z",
    "lte": "2024-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
import json
import pprint
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

image_ids = [feature['id'] for feature in geojson['features']]

# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

In [ ]:
PLANET_TILE_URL_TEMPLATE = "https://tiles.planet.com/data/v1/{item_type}/{item_id}/{{z}}/{{x}}/{{y}}.png?api_key={API_KEY}"

cntd = 0 

for result in geojson['features']:
    try:
        item_id = result["id"]
        item_type = result["properties"]["item_type"]
        
        # Cria URL do tile
        tile_url = PLANET_TILE_URL_TEMPLATE.format(
            item_id=item_id,
            item_type=item_type,
            API_KEY=API_KEY
        )
        
        # Adiciona camada temporária
        added_layer = m.addDataFromPath(tile_url)
        print(f"Processando: {item_id}")
        time.sleep(2)  # Espera carregamento

    except Exception as e:
        print(f"Erro no item {item_id}: {str(e)}")